# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,37.76,84,100,0.00,US,1675826168
1,1,jamestown,42.0970,-79.2353,37.04,93,100,13.80,US,1675826085
2,2,ostrovnoy,68.0531,39.5131,30.56,77,9,22.59,RU,1675826170
3,3,new norfolk,-42.7826,147.0587,62.10,32,5,4.54,AU,1675826171
4,4,camopi,3.1655,-52.3158,71.65,99,100,1.79,GF,1675826173


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
hvplot_map = city_data_df.hvplot.points("Lng", "Lat", geo=True, height=500, width=800, tiles="OSM", clim=(0, 100), size="Humidity", color="City", hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed"], title="City Latitude vs. Humidity (08/01/20) (Hover for more info)")

# Display the map
# YOUR CODE HERE
hvplot_map  

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Max Temp,Cloudiness,Wind Speed)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [32]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df_clean = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Cloudiness"] == 0), :]

# Drop any rows with null values
city_data_df_clean = city_data_df_clean.dropna()

# Display sample data
city_data_df_clean

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,busselton,-33.6500,115.3333,78.98,44,0,1.61,AU,1675826179
26,26,arica,-18.4750,-70.3042,74.75,78,0,5.75,CL,1675826194
311,311,lazaro cardenas,17.9583,-102.2000,74.17,81,0,3.53,MX,1675826313
327,327,rio cuarto,-33.1307,-64.3499,78.57,57,0,6.91,AR,1675826318
347,347,ixtapa,20.7000,-105.2000,70.83,83,0,0.00,MX,1675826325
488,488,venado tuerto,-33.7456,-61.9688,77.77,32,0,3.96,AR,1675826391
538,538,santana do livramento,-30.8908,-55.5328,74.89,37,0,3.76,BR,1675826439


### Step 3: Create a new DataFrame called `hotel_df`.

In [34]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df_clean.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,busselton,-33.6500,115.3333,78.98,44,0,1.61,AU,1675826179,
26,26,arica,-18.4750,-70.3042,74.75,78,0,5.75,CL,1675826194,
311,311,lazaro cardenas,17.9583,-102.2000,74.17,81,0,3.53,MX,1675826313,
327,327,rio cuarto,-33.1307,-64.3499,78.57,57,0,6.91,AR,1675826318,
347,347,ixtapa,20.7000,-105.2000,70.83,83,0,0.00,MX,1675826325,


In [38]:
Response= requests.get(f"https://api.geoapify.com/v1/geocode/reverse?lat={-33.6500}&lon={115.3333}&apiKey={geoapify_key}").json()
Response

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [115.333333, -33.650107]},
   'properties': {'country_code': 'au',
    'housenumber': '29',
    'street': 'Thomas Street',
    'country': 'Australia',
    'county': 'Busselton',
    'datasource': {'sourcename': 'openaddresses',
     'attribution': '© OpenAddresses contributors',
     'license': 'BSD-3-Clause License'},
    'postcode': '6280',
    'state': 'Western Australia',
    'district': 'West Busselton',
    'city': 'West Busselton',
    'lon': 115.333333,
    'lat': -33.650107,
    'state_code': 'WA',
    'distance': 12.28372252753991,
    'result_type': 'building',
    'formatted': '29 Thomas Street, West Busselton WA 6280, Australia',
    'address_line1': '29 Thomas Street',
    'address_line2': 'West Busselton WA 6280, Australia',
    'timezone': {'name': 'Australia/Perth',
     'offset_STD': '+08:00',
     'offset_STD_seconds': 28800,
     'offset_DST': '+08:00',
  

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [46]:
# Set parameters to search for a hotel
radius = 100000
params = {
    "radius": radius,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"within_circle:{latitude},{longitude},{radius}"
    params["bias"] = f"point:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_json = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: No hotel found
arica - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
rio cuarto - nearest hotel: No hotel found
ixtapa - nearest hotel: No hotel found
venado tuerto - nearest hotel: No hotel found
santana do livramento - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
11,11,busselton,-33.6500,115.3333,78.98,44,0,1.61,AU,1675826179,No hotel found
26,26,arica,-18.4750,-70.3042,74.75,78,0,5.75,CL,1675826194,No hotel found
311,311,lazaro cardenas,17.9583,-102.2000,74.17,81,0,3.53,MX,1675826313,No hotel found
327,327,rio cuarto,-33.1307,-64.3499,78.57,57,0,6.91,AR,1675826318,No hotel found
347,347,ixtapa,20.7000,-105.2000,70.83,83,0,0.00,MX,1675826325,No hotel found
488,488,venado tuerto,-33.7456,-61.9688,77.77,32,0,3.96,AR,1675826391,No hotel found
538,538,santana do livramento,-30.8908,-55.5328,74.89,37,0,3.76,BR,1675826439,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [49]:
%%capture --no-display

# Configure the map plot
hotel_hvplot_map= hotel_df.hvplot.points("Lng", "Lat", geo=True, height=500, width=800, tiles="OSM", clim=(0, 100), color="City", hover_cols=["City", "Country", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Hotel Name"], title="City and Hotel (Hover for more info)")

# Display the map
hotel_hvplot_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Max Temp,Humidity,Cloudiness,Wind Speed,Hotel Name)